In [1]:
!pip install numpy matplotlib tensorflow scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.8 MB/s eta 0:00:00m eta 0:00:01
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.0/645.0 MB 4.6 MB/s eta 0:00:00m eta 0:00:010:00:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 7.3 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 3.6 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 7.3 MB/s eta 0:00:000:00:010:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 7.5 MB/s eta 0:00:000:00:010:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.1 MB/s eta 0:00:005.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 7.3 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np

class NeuralNetwork:
    def __init__(self, num_inputs, num_hidden_neurons, num_output_neurons, learning_rate, activation_function_name):
        self.num_inputs = num_inputs
        self.num_hidden_neurons = num_hidden_neurons
        self.num_output_neurons = num_output_neurons
        self.learning_rate = learning_rate

        # Inicializa pesos e biases aleatoriamente
        self.weights_input_hidden = np.random.uniform(-1, 1, (self.num_inputs, self.num_hidden_neurons))
        self.bias_hidden = np.random.uniform(-1, 1, (1, self.num_hidden_neurons))
        self.weights_hidden_output = np.random.uniform(-1, 1, (self.num_hidden_neurons, self.num_output_neurons))
        self.bias_output = np.random.uniform(-1, 1, (1, self.num_output_neurons))

        # Define a função de ativação e sua derivada
        self.activation_function, self.derivative_activation_function = self._get_activation_function(activation_function_name)

    def _get_activation_function(self, name):
        if name == 'sigmoid':
            return self._sigmoid, self._sigmoid_derivative
        elif name == 'tanh':
            return self._tanh, self._tanh_derivative
        elif name == 'relu':
            return self._relu, self._relu_derivative
        else:
            raise ValueError("Função de ativação não suportada. Escolha entre 'sigmoid', 'tanh', 'relu'.")

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _sigmoid_derivative(self, x):
        return x * (1 - x) # x aqui é a saída da função sigmoid

    def _tanh(self, x):
        return np.tanh(x)

    def _tanh_derivative(self, x):
        return 1 - (x ** 2) # x aqui é a saída da função tanh

    def _relu(self, x):
        return np.maximum(0, x)

    def _relu_derivative(self, x):
        return (x > 0).astype(float) # x aqui é a saída da função relu

    def feedforward(self, inputs):
        # Camada oculta
        self.hidden_layer_input = np.dot(inputs, self.weights_input_hidden) + self.bias_hidden
        self.hidden_layer_output = self.activation_function(self.hidden_layer_input)

        # Camada de saída
        self.output_layer_input = np.dot(self.hidden_layer_output, self.weights_hidden_output) + self.bias_output
        self.predicted_output = self.activation_function(self.output_layer_input)
        return self.predicted_output

    def backpropagate(self, inputs, targets):
        # Calcular o erro na camada de saída
        error_output = targets - self.predicted_output

        # Gradiente da camada de saída
        # É importante usar a saída da função de ativação para calcular a derivada,
        # e não a entrada líquida (net_o)
        delta_output = error_output * self.derivative_activation_function(self.predicted_output)

        # Calcular o erro na camada oculta
        error_hidden = np.dot(delta_output, self.weights_hidden_output.T)

        # Gradiente da camada oculta
        delta_hidden = error_hidden * self.derivative_activation_function(self.hidden_layer_output)

        # Atualizar pesos e biases da camada oculta para saída
        self.weights_hidden_output += self.learning_rate * np.dot(self.hidden_layer_output.T, delta_output)
        self.bias_output += self.learning_rate * np.sum(delta_output, axis=0, keepdims=True)

        # Atualizar pesos e biases da camada de entrada para oculta
        self.weights_input_hidden += self.learning_rate * np.dot(inputs.T, delta_hidden)
        self.bias_hidden += self.learning_rate * np.sum(delta_hidden, axis=0, keepdims=True)

    def train(self, training_data, epochs):
        for epoch in range(epochs):
            total_error = 0
            for inputs, targets in training_data:
                inputs = np.array([inputs]) # Garante que as entradas sejam uma matriz 2D
                targets = np.array([targets]) # Garante que os targets sejam uma matriz 2D

                self.feedforward(inputs)
                self.backpropagate(inputs, targets)

                total_error += np.mean(np.abs(targets - self.predicted_output)) # MAE como métrica de erro
            # print(f"Época {epoch+1}/{epochs}, Erro: {total_error / len(training_data):.4f}")

    def predict(self, inputs):
        inputs = np.array([inputs]) # Garante que as entradas sejam uma matriz 2D
        return self.feedforward(inputs)

# --- Funções para gerar dados AND, OR, XOR ---
def generate_boolean_data(num_inputs, gate_type):
    data = []
    # Itera sobre todas as combinações possíveis de entradas booleanas
    for i in range(2**num_inputs):
        binary_representation = bin(i)[2:].zfill(num_inputs)
        inputs = [int(bit) for bit in binary_representation]
        
        target = 0
        if gate_type == 'AND':
            target = 1 if all(inputs) else 0
        elif gate_type == 'OR':
            target = 1 if any(inputs) else 0
        elif gate_type == 'XOR':
            target = sum(inputs) % 2 # XOR é 1 se o número de 1s for ímpar
        else:
            raise ValueError("Tipo de porta lógica inválido. Escolha 'AND', 'OR' ou 'XOR'.")
        
        data.append((inputs, [target])) # O target também precisa ser uma lista para numpy
    return data


In [3]:

# --- Experimentos ---

def run_experiment(gate_type, num_inputs, learning_rate, num_hidden_neurons, activation_function_name, epochs=10000):
    print(f"\n--- Experimentando: {gate_type} com {num_inputs} entradas ---")
    print(f"Taxa de Aprendizado: {learning_rate}, Neurônios Ocultos: {num_hidden_neurons}, Função de Ativação: {activation_function_name}")

    training_data = generate_boolean_data(num_inputs, gate_type)
    nn = NeuralNetwork(num_inputs, num_hidden_neurons, 1, learning_rate, activation_function_name)
    nn.train(training_data, epochs)

    print("\nResultados do Teste:")
    correct_predictions = 0
    total_predictions = 0
    for inputs, target in training_data:
        prediction = nn.predict(inputs)[0][0]
        # Para saídas booleanas, arredondamos para 0 ou 1
        predicted_class = 1 if prediction >= 0.5 else 0 
        
        print(f"Entrada: {inputs}, Saída Esperada: {target[0]}, Saída Prevista: {prediction:.4f} (Classe: {predicted_class})")
        if predicted_class == target[0]:
            correct_predictions += 1
        total_predictions += 1
    
    accuracy = (correct_predictions / total_predictions) * 100
    print(f"Acurácia: {accuracy:.2f}%")
    return accuracy

# 1) A importância da taxa de aprendizado
print("\n--- Investigando a importância da Taxa de Aprendizado ---")
# Para AND com 2 entradas, 4 neurônios ocultos, sigmoide
run_experiment('AND', 2, 0.1, 4, 'sigmoid') # Taxa de aprendizado padrão
run_experiment('AND', 2, 0.01, 4, 'sigmoid') # Taxa de aprendizado menor
run_experiment('AND', 2, 0.5, 4, 'sigmoid')  # Taxa de aprendizado maior (pode oscilar)

# 2) A importância do bias
# O bias já está incluído na implementação da NeuralNetwork, 
# pois ele é crucial para redes neurais. 
# Para demonstrar sua importância, poderíamos criar uma versão sem bias, 
# mas isso geralmente leva a uma incapacidade de aprender.
# A melhor forma de "investigar" é entender que sem ele, a rede não conseguiria 
# deslocar a função de ativação, o que é vital para separar dados não linearmente separáveis.

# 3) A importância da função de ativação
print("\n--- Investigando a importância da Função de Ativação ---")
# Para XOR com 2 entradas, 4 neurônios ocultos (XOR é o melhor para ver a diferença)
run_experiment('XOR', 2, 0.1, 4, 'sigmoid')
run_experiment('XOR', 2, 0.1, 4, 'tanh')
# RELU pode ter problemas de "dying ReLU" em alguns cenários e requer mais cuidado com a inicialização,
# mas vamos testar para fins de demonstração.
run_experiment('XOR', 2, 0.1, 4, 'relu') 

# Testes com diferentes números de entradas
print("\n--- Testando com diferentes números de entradas ---")
run_experiment('AND', 3, 0.1, 8, 'sigmoid') # AND com 3 entradas
run_experiment('OR', 4, 0.1, 8, 'sigmoid')  # OR com 4 entradas
run_experiment('XOR', 3, 0.1, 8, 'tanh')   # XOR com 3 entradas (mais complexo)


--- Investigando a importância da Taxa de Aprendizado ---

--- Experimentando: AND com 2 entradas ---
Taxa de Aprendizado: 0.1, Neurônios Ocultos: 4, Função de Ativação: sigmoid

Resultados do Teste:
Entrada: [0, 0], Saída Esperada: 0, Saída Prevista: 0.0001 (Classe: 0)
Entrada: [0, 1], Saída Esperada: 0, Saída Prevista: 0.0237 (Classe: 0)
Entrada: [1, 0], Saída Esperada: 0, Saída Prevista: 0.0228 (Classe: 0)
Entrada: [1, 1], Saída Esperada: 1, Saída Prevista: 0.9625 (Classe: 1)
Acurácia: 100.00%

--- Experimentando: AND com 2 entradas ---
Taxa de Aprendizado: 0.01, Neurônios Ocultos: 4, Função de Ativação: sigmoid

Resultados do Teste:
Entrada: [0, 0], Saída Esperada: 0, Saída Prevista: 0.0264 (Classe: 0)
Entrada: [0, 1], Saída Esperada: 0, Saída Prevista: 0.2020 (Classe: 0)
Entrada: [1, 0], Saída Esperada: 0, Saída Prevista: 0.2024 (Classe: 0)
Entrada: [1, 1], Saída Esperada: 1, Saída Prevista: 0.7131 (Classe: 1)
Acurácia: 100.00%

--- Experimentando: AND com 2 entradas ---
Taxa de 

100.0